In [1]:
!pip install datasets accelerate wandb peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 5.1 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 4.7 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 4.9 MB/s eta 0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 5.2 MB/s eta 0:00:006.0 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 5.6 MB/s eta 0:00:006.5 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.4 MB/s eta 0:00:00


In [2]:
from wandb import login

login("never", "***REMOVED***")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/factory/.netrc


True

In [3]:
!huggingface-cli login --token ***REMOVED***

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/factory/.cache/huggingface/token
Login successful


In [4]:
%%bash

git clone https://github.com/huggingface/diffusers.git
cd diffusers
pip install -e ".[torch]"

Cloning into 'diffusers'...


Obtaining file:///home/factory/work/mb/factory/train/diffusers
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
Checking if build backend supports build_editable: started
Checking if build backend supports build_editable: finished with status 'done'
  Building editable for diffusers (pyproject.toml): started
  Building editable for diffusers (pyproject.toml): finished with status 'done'
  Created wheel for diffusers: filename=diffusers-0.28.0.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
factory 0.0.0a0 requires diffusers<0.28.0,>=0.27.2, but you have diffusers 0.28.0.dev0 which is incompatible.


In [5]:
!scp diffusers/examples/dreambooth/train_dreambooth_lora.py train_dreambooth_lora.py

In [6]:
!mkdir -p models/

In [11]:
!accelerate launch train_dreambooth_lora.py \
  --pretrained_model_name_or_path='segmind/tiny-sd'  \
  --instance_data_dir='./datasets/margot_robbie' \
  --output_dir='./models/margot_robbie-full-lora' \
  --instance_prompt="MAGROB style" \
  --class_data_dir='./datasets/image_data' \
  --class_prompt='style' \
  --with_prior_preservation \
  --num_class_images=10 \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --checkpointing_steps=100 \
  --learning_rate=1e-4 \
  --report_to="wandb" \
  --lr_scheduler="cosine_with_restarts" \
  --lr_warmup_steps=0 \
  --max_train_steps=500 \
  --validation_prompt="MAGROB style woman on a field of flowers" \
  --validation_epochs=5 \
  --seed="0" \
  --push_to_hub \
  --hub_model_id "philipp-zettl/margot_robbie-lora"

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
04/07/2024 14:13:47 - INFO - __main__ - Distributed environment: DistributedType.NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no

You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
{'rescale_betas_zero_snr'} was not found in config. Values will be initialized to default values.
{'latents_std', 'latents_mean'} was not found in config. Values will be initialized to default values.
{'cross_attention_norm', 'time_embedding_type', 'use_linear_projection', 'only_

In [ ]:
!rm -rf train_lora.py

In [4]:
from train_dreambooth_lora import *

pretrained_model_name_or_path = 'segmind/tiny-sd'
revision = None
variant = None
weight_dtype = torch.float16
output_dir = './models/margot_robbie-full-lora/'
logging_dir = 'logs/'

In [5]:
def log_validation(
    pipeline,
    args,
    accelerator,
    pipeline_args,
    epoch,
    is_final_validation=False,
):
    logger.info(
        f"Running validation... \n Generating {args.num_validation_images} images with prompt:"
        f" {args.validation_prompt}."
    )
    # We train on the simplified learning objective. If we were previously predicting a variance, we need the scheduler to ignore it
    scheduler_args = {}

    if "variance_type" in pipeline.scheduler.config:
        variance_type = pipeline.scheduler.config.variance_type

        if variance_type in ["learned", "learned_range"]:
            variance_type = "fixed_small"

        scheduler_args["variance_type"] = variance_type

    pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config, **scheduler_args)

    pipeline = pipeline.to(accelerator.device)
    pipeline.set_progress_bar_config(disable=True)

    # run inference
    generator = torch.Generator(device=accelerator.device).manual_seed(args.seed) if args.seed else None

    images = []
    for _ in range(args.num_validation_images):
        with torch.cuda.amp.autocast():
            image = pipeline(**pipeline_args, generator=generator).images[0]
            images.append(image)

    for tracker in accelerator.trackers:
        phase_name = "test" if is_final_validation else "validation"
        if tracker.name == "tensorboard":
            np_images = np.stack([np.asarray(img) for img in images])
            tracker.writer.add_images(phase_name, np_images, epoch, dataformats="NHWC")
        if tracker.name == "wandb":
            tracker.log(
                {
                    phase_name: [
                        wandb.Image(image, caption=f"{i}: {args.validation_prompt}") for i, image in enumerate(images)
                    ]
                }
            )

    del pipeline
    torch.cuda.empty_cache()

    return images

In [6]:
class FooArgs:
    num_validation_images = 10
    seed = 0

In [8]:
pipeline = DiffusionPipeline.from_pretrained(
    pretrained_model_name_or_path, revision=revision, variant=variant, torch_dtype=weight_dtype
)

# load attention processors
pipeline.load_lora_weights(output_dir, weight_name="pytorch_lora_weights.safetensors")

logging_dir = Path(output_dir, logging_dir)
accelerator_project_config = ProjectConfiguration(project_dir=output_dir, logging_dir=logging_dir)
accelerator = Accelerator(
    gradient_accumulation_steps=1,
    mixed_precision='fp16',
    log_with='wandb',
    project_config=accelerator_project_config,
)
# run inference
images = []
validation_prompt = 'MAGROB style a woman in a field'

if validation_prompt:
    pipeline_args = {"prompt": validation_prompt, "num_inference_steps": 25}
    images = log_validation(
        pipeline,
        FooArgs(),
        accelerator,
        pipeline_args,
        500,
        is_final_validation=True,
    )

vae/diffusion_pytorch_model.safetensors not found


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [9]:
pipeline = DiffusionPipeline.from_pretrained(
    'segmind/tiny-sd',
    torch_dtype=torch.float16,
    safety_checker=None,
    revision=None
)
save_model_card(
    "philipp-zettl/margot_robbie-lora",
    images=images,
    base_model='segmind/tiny-sd',
    train_text_encoder=False,
    prompt="MAGROB style",
    repo_folder='./models/margot_robbie-full-lora/',
    pipeline=pipeline,
)
upload_folder(
    repo_id='philipp-zettl/margot_robbie-lora',
    folder_path='./models/margot_robbie-full-lora/',
    commit_message="End of training",
    ignore_patterns=["step_*", "epoch_*"],
)

vae/diffusion_pytorch_model.safetensors not found


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

scheduler.bin:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Upload 21 LFS files:   0%|          | 0/21 [00:00<?, ?it/s]

random_states_0.pkl:   0%|          | 0.00/14.3k [00:00<?, ?B/s]

...

pytorch_lora_weights.safetensors:   0%|          | 0.00/1.75M [00:00<?, ?B/s]

random_states_0.pkl:   0%|          | 0.00/14.3k [00:00<?, ?B/s]

scheduler.bin:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_lora_weights.safetensors:   0%|          | 0.00/1.75M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/philipp-zettl/margot_robbie-lora/commit/9763b6bc9d090b59e81b579aea7a414eab1ed3c1', commit_message='End of training', commit_description='', oid='9763b6bc9d090b59e81b579aea7a414eab1ed3c1', pr_url=None, pr_revision=None, pr_num=None)

## different approach

In [10]:
!scp diffusers/examples/text_to_image/train_text_to_image_lora_sdxl.py ./train_text_to_image_lora_sdxl.py

In [6]:
!accelerate launch train_text_to_image_lora_sdxl.py \
  --pretrained_model_name_or_path="segmind/tiny-sd" \
  --pretrained_vae_model_name_or_path="segmind/tiny-sd" \
  --dataset_name='philipp-zettl/jon_juarez' --caption_column="text" \
  --resolution=1024 --random_flip \
  --train_batch_size=1 \
  --num_train_epochs=2 --checkpointing_steps=500 \
  --learning_rate=1e-04 --lr_scheduler="constant" --lr_warmup_steps=0 \
  #--mixed_precision="fp16" \
  --seed=42 \
  --output_dir="tiny-sd-jon_juarez-lora" \
  --validation_prompt="cute dragon creature" --report_to="wandb" \
  --push_to_hub \
  --hub_model_id "philipp-zettl/sd-jon_juarez-model-lora-ssd"


The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
/home/factory/.cache/pypoetry/virtualenvs/factory-HxwD1Xz5-py3.11/lib/python3.11/site-packages/accelerate/accelerator.py:391: UserWarning: `log_with=tensorboard` was passed but no supported trackers are currently installed.
  warnings.warn(f"`log_with={log_with}` was passed but no supported trackers are currently installed.")
04/07/2024 16:39:25 - INFO - __main__ - Distributed environment: DistributedType.NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no

Traceback (most recent call last):
  File "/home/factory/.cache/pypoetry/virtualenvs/factory-HxwD1Xz5-py3.11

In [7]:
!accelerate launch train_dreambooth_lora.py \
  --pretrained_model_name_or_path='segmind/SSD-1B'  \
  --instance_data_dir='./datasets/margot_robbie' \
  --output_dir='./models/ssd-margot_robbie-full-lora' \
  --instance_prompt="MAGROB style" \
  --class_data_dir='./datasets/image_data' \
  --class_prompt='style' \
  --with_prior_preservation \
  --num_class_images=10 \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --checkpointing_steps=100 \
  --learning_rate=5e-4 \
  --report_to="wandb" \
  --lr_scheduler="polynomial" \
  --lr_warmup_steps=0 \
  --max_train_steps=500 \
  --validation_prompt="MAGROB style woman on a field of flowers" \
  --validation_epochs=5 \
  --seed="0" \
  --push_to_hub \
  --hub_model_id "philipp-zettl/ssd-margot_robbie-lora"

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
04/07/2024 16:42:18 - INFO - __main__ - Distributed environment: DistributedType.NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no

You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
{'rescale_betas_zero_snr', 'variance_type', 'dynamic_thresholding_ratio', 'thresholding', 'clip_sample_range'} was not found in config. Values will be initialized to default values.
vae/config.json: 100%|█████████████████████████| 602/602 [00:00<00:00, 2.16MB/s]
vae/diffusion_pyt

## Next attempt

In [10]:
!scp diffusers/examples/text_to_image/train_text_to_image_lora.py .

In [12]:
from accelerate.utils import write_basic_config

write_basic_config()

PosixPath('/home/factory/.cache/huggingface/accelerate/default_config.yaml')

In [ ]:
!accelerate launch train_text_to_image_lora.py \
  --pretrained_model_name_or_path="segmind/tiny-sd" \
  --dataset_name="philipp-zettl/nsfw-images" \
  --dataloader_num_workers=1 \
  --resolution=512 \
  --center_crop \
  --random_flip \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --max_train_steps=15000 \
  --learning_rate=1e-04 \
  --max_grad_norm=1 \
  --lr_scheduler="cosine" \
  --lr_warmup_steps=0 \
  --output_dir="./finetune/lora/nsfw" \
  --push_to_hub \
  --hub_model_id="philipp-zettl/tiny-sd-nsfw-lora" \
  --report_to=wandb \
  --checkpointing_steps=500 \
  --validation_prompt="SUPER_PROMPT a nude woman on the beach" \
  --seed=1337

04/07/2024 17:31:55 - INFO - __main__ - Distributed environment: DistributedType.NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no

{'rescale_betas_zero_snr'} was not found in config. Values will be initialized to default values.
{'latents_mean', 'latents_std'} was not found in config. Values will be initialized to default values.
{'resnet_out_scale_factor', 'cross_attention_norm', 'addition_embed_type', 'num_class_embeds', 'attention_type', 'addition_embed_type_num_heads', 'use_linear_projection', 'projection_class_embeddings_input_dim', 'encoder_hid_dim_type', 'addition_time_embed_dim', 'resnet_time_scale_shift', 'time_embedding_dim', 'conv_out_kernel', 'mid_block_scale_factor', 'dual_cross_attention', 'time_cond_proj_dim', 'upcast_attention', 'only_cross_attention', 'transformer_layers_per_block', 'class_embed_type', 'time_embedding_type', 'conv_in_kernel', 'mid_block_only_cross_attention', 'num_attention_heads', 'reverse_transformer_